# Notebook 1: Examples of using Paicos

This script shows how to

- load data
- make projections and slices
- save them as an 'ArepoImage'
- how to convert the Arepo data from comoving code units to physical values in various unit systems


## Compilation
The first step is to compile the code, this only needs to be done the first time you use Paicos and if you have not already followed the installation instructions (replace the path to your own cloned version, you will also need to add this path to your PYTHONPATH).

In [ ]:
#%%bash
#cd ~/projects/paicos
#make clean
#make

## Loading arepo snapshots

We load a zoom-in simulation of a galaxy cluster simulation below (the data is not included in the repo, so you will need to download this. See documentation)

In [ ]:
import paicos as pa
import numpy as np

# A snapshot object
snap = pa.Snapshot(pa.data_dir, 247)

# The center of the most massive Friends-of-friends group in the simulation
center = snap.Cat.Group['GroupPos'][0]

## Useful metadata
We can look at some of snap attributes:

In [ ]:
# Age of the Universe for the snapshot
snap.age

In [ ]:
# The lookback time
snap.lookback_time

In [ ]:
# An astropy cosmology object (used internally to calculate the age and lookback, 
# cosmological parameters automatically loaded from the the snapshot)
snap.cosmo

In [ ]:
# The size of the computational box
snap.box_size.to('Mpc').to_physical

In [ ]:
# The redshift
snap.z

In [ ]:
# Adiabatic index used in the simulation
snap.gamma

In [ ]:
# Contents of the three hdf5 groups containing information about the snapshot (uncomment to see output)
#snap.Header
#snap.Parameters
#snap.Config

## Loading of data blocks
Loading of data can be done using function calls or by trying to access them explicitly.
Here we load the Arepo data as a PaicosQuantity (basically a subclass of an astropy quantity), which gives the numeric data units and some useful methods. The numeric values are the same as stored in the hdf5 files but we can now see the units used in the simulation. Here small_h and small_a are the reduced Hubble parameter and the scale factor, respectively. 

In [ ]:
# Load some variables from the PartType 0 (gas variables) 

# You can explicitly load using function call:
snap.load_data(0, 'Coordinates')
snap['0_Coordinates']

# But is much easier to just do it like this:
snap['0_Density']
snap['0_MagneticField']

# snap
snap['0_Volume']

In [ ]:
# The available fields for a PartType can be found as shown below for parttype 0 (the gas)
keys = snap.info(0)

# alternatively, by starting to type and using tab-completion, i.e., snap['0_  and then hit tab

## Unit conversion 
Here we show how to use convert the density field to various useful physical units and
how to get rid of the a and h factors used in cosmological simulations with Arepo.

In [ ]:
# Unit conversion
rho = snap['0_Density']
print('rho[0] in CGS:\t', rho[0].cgs)
print('rho[0] in SI:\t', rho[0].si)
print("rho[0] in 'astro' units:\t", rho[0].astro)
print("rho[0] in Msun/au^3:\t", rho[0].to('Msun/au3'), '\n\n')

# Get rid of h factors
print('rho[0] without h:\t', rho[0].no_small_h)

# Get rid of both a and h factors
print('rho[0] without a and h:\t', rho[0].to_physical, '\n\n')

# Get a label for use in plots
print(rho.label(r'\rho'))
print(rho.astro.label(r'\rho'))

### Changing the units in a PaicosQuantity
Please note that the methods above return a new object without modifying the original data. Modification can be done by overwriting, e.g., like this:

In [ ]:
rho = rho.to_physical
rho

### Getting rid of units

The PaicosQuantity uses the astropy quantity internally, which again uses numpy arrays.
In case you are not familiar with astropy: Here is how you can access the unit and numeric values independently like this: 

In [ ]:
# The unit
rho.unit

In [ ]:
# The numeric values (a numpy array)
rho.value

## Automatically computing derived variables
Here we show how Paicos can automatically compute derived variables.
Paicos gives information about what is happening under the hood

This feature can be turned off by setting

```
pa.settings.use_only_user_functions = True
```
but we note that changes to `pa.settings.use_only_user_functions` only take effect for freshly loaded snapshots.

In [ ]:
# The gas cell volumes (per default Arepo only outputs the gas mass and density)
snap['0_Volume']

# The gas temperature times the cell gas mass
snap['0_TemperaturesTimesMasses']

# The gas enstrophy
snap['0_Enstrophy']

The console output can be turned off by setting
```
pa.settings.print_info_when_deriving_variables = False
```
This is illustrated below where no information is printed:

In [ ]:
# Turn off info
pa.settings.print_info_when_deriving_variables = False

# The metallicity multiplied by the mass
snap['0_GFM_MetallicityTimesMasses']

# Turn info back on
pa.settings.print_info_when_deriving_variables = True

## Making projections

We now use the Paicos projector class, the 'widths' vector is the size of the considered box in x,y,z coordinates. This box is centered at 'center' vector.

The direction can be set to 'x', 'y' or 'z'. If the direction is 'z' (as below) then widths[2] is the depth of the projection and the 2D returned array is in the xy plane.

npix is the number of pixels in the horizontal direction of the image. The width/height ratio should be such that $$npix*height/width$$ is an integer, such that the image pixels are square.

In [ ]:
# The widths of the subbox to be projected
widths = [26000, 13000, 10000]

# Create a projector object
projector = pa.Projector(snap, center, widths, 'z', npix=2048)

# Let's look at its docstring
projector?

### Calling project_variable
We can call the project_variable method as below. This method can take a number of standard strings (which then internally calls the get_variable function, see further details below) or it can take an array. Both methods are shown below.

In [ ]:
Masses = projector.project_variable('0_Masses')
Volumes = projector.project_variable(snap['0_Volume'])
rho = Masses/Volumes

rho

We can now plot the projected density

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
plt.rc('image', origin='lower', cmap='RdBu_r', interpolation='None')
plt.imshow(rho.value, cmap='YlGnBu', extent=projector.extent.value, norm=LogNorm())

The projector object contains a number of useful attributes with mostly self-explanatory names:

In [ ]:
# Widths (same as user input)
projector.widths

# Volume of the subbox
projector.volume

# Area per pixel
projector.area_per_pixel

# Volume per pixel
projector.volume_per_pixel

# Center of the image (same as user input)
projector.center

# Depth of the projection
projector.depth

# Height of the image (i.e. along the vertical direction of the image)
projector.height

# Width of the image (i.e. along the horizontal direction of the image)
projector.width

# For use in the matplotlib argument extent
projector.extent

# For centering the image such that its center is at (0, 0).
projector.centered_extent 

# E.g. a centered image is created like this
plt.imshow(rho.value, cmap='YlGnBu', extent=projector.centered_extent.value, norm=LogNorm())


## Making slices

Next, we will take a look at making a slice through the simulation. The width is by definition zero, and the user has to set this explicitly by setting a zero in the 'widths' vector. Below we show a slice of density, comparing with the projected density.

In [ ]:
widths = [26000, 13000, 0]
slicer = pa.Slicer(snap, center, widths, 'z', npix=2048)

In [ ]:
plt.figure(1)
plt.clf()
fix, axes = plt.subplots(nrows=2)

# Slice by passing an array
rho_slice = slicer.slice_variable(snap['0_Density'])

# Slice by passing a string (see snap.info(0) for the available strings)
rho_slice = slicer.slice_variable('0_Density')

# Now plot slice and projection next to each other
axes[0].imshow(rho_slice.to_physical.value, norm=LogNorm())
axes[1].imshow(rho.value, norm=LogNorm())
axes[0].set_title('Slice')
axes[1].set_title('Projected')
for ii in range(2):
    axes[ii].set_axis_off()
# plt.savefig('halo3_Z12_slice_projec_comparison.pdf', dpi=2000, bbox_inches='tight')

We can also make slices of other variables. The Slicer object stores the required information (indices of the Voronoi cells closest to the image grid points), so the computing time needed for making additional slices is neglibible.

Let us for instance consider the enstrophy which gives an indication of the amount of turbulence in the galaxy cluster.
It is defined as

1/2|∇×v|²

and can be found from the 'VelocityGradient' field (the 3x3 tensor of velocity derivatives, ∂ᵢvⱼ, which is stored in the example Arepo snapshot). This is done internally below:

In [ ]:
extent = slicer.extent.to('Mpc')

plt.imshow(slicer.slice_variable('0_Enstrophy').value,
           extent=extent.value,
           norm=LogNorm())

## Storing image data

The computing time for slices, and in particular, projections, is often quite long. It is therefore convenient to be able to store the image data so that this step is de-coupled from the often many matplotlib iterations.

Below we illustrate how to save an Arepo image, created using either a Projector or Slicer object.

In [ ]:
image_file = pa.ArepoImage(slicer, basedir=pa.data_dir,
                           basename='test_arepo_image_format')

image_file.save_image('Density', slicer.slice_variable('0_Density'))
image_file.save_image('Enstrophy', slicer.slice_variable('0_Enstrophy'))

image_file.finalize()

The constructed file is found at:

In [ ]:
image_file.filename

Now lets open this image and looks at its contents:

In [ ]:
import h5py
f = h5py.File(image_file.filename, 'r')

In [ ]:
list(f.keys())

Here 'Config', 'Header', 'Parameters' are groups copied over from the snapshot file used to create the image (.0.hdf5 when there are multiple files). 'Density' and 'Enstrophy' are 2D arrays with the saved images. The group 'image_info' contains essential information about the image, namely:

In [ ]:
print(f['image_info'].keys())
print(f['image_info'].attrs.keys())

We can plot the image and use 'image_info' to get the extent of the image.

In [ ]:
im = plt.imshow(f['Density'], extent=f['image_info']['extent'], norm=LogNorm())
cbar = plt.colorbar(im, fraction=0.025, pad=0.04)

## Getting the units right

The plot above is still in comoving code units, we can use the ImageReader class to automatically get the image data in the form of PaicosQuantities (i.e. with units and in-built methods for manipulation). All the relevant information is stored in the image file, e.g.:

In [ ]:
dict(f['Density'].attrs)

In [ ]:
im = pa.ImageReader(basedir=pa.data_dir, snapnum=247,
                 basename='test_arepo_image_format')

# Convert the extent to Mpc and get rid of the h factor
extent = im.centered_extent.to('Mpc').no_small_h #.to_physical

# Get rid of both a and h in the density
rho = im['Density'].to_physical

# Convert rho to typical astro units
rho = rho.astro

# Convert rho to cgs
rho = rho.cgs

# Plot the image
image = plt.imshow(rho.value, extent=extent.value, norm=LogNorm())

# Add a colorbar
cbar = plt.colorbar(image, fraction=0.025, pad=0.04)

# Set the labels. The units for the labels are here set using the .label method
# of the PaicosQuantity. This internally uses astropy functionality and is
# mainly a convenience function.
cbar.set_label(rho.label('\\rho'))
plt.xlabel(extent.label('x'))
plt.ylabel(extent.label('y'))

In [ ]:
im.center
im.widths